In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

C:\Users\dgarb\anaconda3_reinstall\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Classification function

In [2]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score, roc_auc_score

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import StratifiedKFold


from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy import stats
from sklearn.metrics import accuracy_score



In [3]:
def class_rs(
#     model=  LogisticRegression(solver = 'saga', class_weight='balanced', max_iter=500),
model = None,
x_train = None,
x_test =  None,
y_train = None,
y_test =  None,
params = None,
    # params = {'penalty':['l2'],'C':[0.0001, 0.01]} # ridge
#     params = {'penalty':['l1'],'C':[0.0001, 0.01]} # lasso
    # params = {'penalty':['elasticnet'],'C':[0.0001, 0.01], 'l1_ratio':[0.5]}  # elastic net
splits = 5,
scoring = 'roc_auc',
logistic_reg = None,
tree_based = None):
    ###############################################################################
    
   
    features = list(x_train.columns)

    ss = StandardScaler()
    x_train = pd.DataFrame(ss.fit_transform(x_train), columns = features)
    x_test = pd.DataFrame(ss.fit_transform(x_test), columns = features)

    skfold = StratifiedKFold(n_splits= splits, shuffle=True, random_state=0)

    rs = RandomizedSearchCV(model, params, cv=skfold,  return_train_score=True, refit=True, scoring=scoring)
    rs.fit(x_train,y_train)
    rs.best_params_

    class_dict = {}
    model= rs.best_estimator_
    class_dict['mod'] = model
    ###################################    

    print('best params: ',rs.best_params_)
    class_dict['best params'] = rs.best_params_

    print('score: ',rs.score(x_train,y_train))
    class_dict['score'] = rs.score(x_train,y_train)

    print('  ')
    print('test score: ',rs.score(x_test,y_test))
    class_dict['test score'] = rs.score(x_test,y_test)

    from sklearn.metrics import confusion_matrix

    train_pred = rs.best_estimator_.predict(x_train)

    print('confusion matrix train')
    print(confusion_matrix(y_train, train_pred))
    
    class_dict['confusion_matrix_train'] = confusion_matrix(y_train, train_pred)
    print(' ')
    
    test_pred = rs.best_estimator_.predict(x_test)
    print('confusion matrix test')
    print(confusion_matrix(y_test, test_pred))
    class_dict['confusion_matrix_test'] = confusion_matrix(y_test, test_pred)
    print(' ')
    


   
    print(' ')
    
    print("train roc_auc_score: ", roc_auc_score(y_train, rs.predict_proba(x_train)[:,1]))
    
    class_dict['train_roc_auc_score'] = roc_auc_score(y_train, rs.predict_proba(x_train)[:,1])
    class_dict['predicted_probabilities_1_train']  = rs.predict_proba(x_train)[:,1]
    
    print(' ')
    
    print("test roc_auc_score: ", roc_auc_score(y_test, rs.predict_proba(x_test)[:,1]))
    
    class_dict['roc_auc_score'] = roc_auc_score(y_test, rs.predict_proba(x_test)[:,1])

    class_dict['predicted_probabilities_1_test'] = rs.predict_proba(x_test)[:,1]
    
    print(' ')
    #calculate KS train_______________________________________________________________________________________________
    
    predprob =list(class_dict['predicted_probabilities_1_train'])

    y = list(y_train)

    data = {'target_train':y,'predprob':predprob}

    df_prob_train= pd.DataFrame(data)
    df_prob_train.head()

    from scipy.stats import ks_2samp
    
    print('Training KS statistic')
    train_ks = ks_2samp(df_prob_train.loc[df_prob_train.target_train==0,"predprob"], df_prob_train.loc[df_prob_train.target_train==1,"predprob"])
    print(train_ks)
    class_dict['train_ks'] = train_ks
    
    print(' ')
    #calculate KS test_______________________________________________________________________________________________
    predprob2 =list(class_dict['predicted_probabilities_1_test'])

    y_ = list(y_test)

    data = {'target_test':y_,'predprob':predprob2}

    df_prob_test= pd.DataFrame(data)
    df_prob_test.head()

    
    print('Test KS statistic')
    test_ks = ks_2samp(df_prob_test.loc[df_prob_test.target_test==0,"predprob"], df_prob_test.loc[df_prob_test.target_test==1,"predprob"])
    print(test_ks)
    class_dict['test_ks'] = test_ks
    print(' ')
    #end calculate KS test_______________________________________________________________________________________________
   

    if logistic_reg == True:

        coefs = pd.Series(model.coef_[0], name = 'coef' )

        varnames = pd.Series(features, name = 'features')


        pd.set_option('display.max_rows', None) # or 1000.

        abs_coefs = pd.Series(np.abs(coefs),name = 'abs_val')

        coefs_ = pd.concat([varnames,coefs, abs_coefs], axis=1)

        # #return:
        model_importances = coefs_.sort_values(by=['abs_val'], ascending=False)
        model_importances.index=(range(model_importances.shape[0]))
        model_importances
        class_dict['model_importances'] = model_importances

        print(model_importances)
        print(' ')


        print('model_importances.shape: ',model_importances.shape)

    elif tree_based == True:
        pd.set_option('display.max_rows', None) # or 1000.

        #return
        varnames = pd.Series(features, name = 'features')

        importances_ = pd.Series(model.feature_importances_, name = 'importances')

        df = pd.concat([varnames,importances_], axis=1)

        model_importances = df.sort_values(by=['importances'], ascending=False)

        model_importances.index=(range(model_importances.shape[0]))

        class_dict['model_importances'] = model_importances

        print(model_importances)



    return class_dict

In [34]:
def cm_infor(mod):
    tn, fp, fn, tp = mod['confusion_matrix_test'].ravel() #from sklearn documentation
    
    recall = tp/(tp +fn)
    specificity = tn / (tn + fp)
    accuracy = (tp+tn)/(tp + tn + fp + fn)
    precision = tp/(tp + fp)
    F1 = 2 * (precision * recall) / (precision + recall)
    
    print('recall = ',recall)
    print('specificity = ',specificity)
    print('accuracy = ',accuracy)
    print('precision = ',precision)
    print('F1 score = ', F1)

label_encode function

In [5]:
def label_encode(df,feature):
    from sklearn.preprocessing import LabelEncoder
    lencoder = LabelEncoder()
    df[feature] = lencoder.fit_transform(df[feature]).reshape((-1,1))

### Import data

In [7]:
X_train_dep = pd.read_csv(r'C:\Users\dgarb\OneDrive\Documents\APZIVA\Project 2\data\X_train_dep.csv',index_col=0)

X_test_dep = pd.read_csv(r'C:\Users\dgarb\OneDrive\Documents\APZIVA\Project 2\data\X_test_dep.csv',index_col=0)

y_train_dep_df = pd.read_csv(r'C:\Users\dgarb\OneDrive\Documents\APZIVA\Project 2\data\y_train_dep_df.csv',index_col=0)

y_test_dep_df  = pd.read_csv(r'C:\Users\dgarb\OneDrive\Documents\APZIVA\Project 2\data\y_test_dep_df.csv',index_col=0)

In [8]:
y_test_dep_df.head()

,y_label
13130,False
11127,True
17899,True
39652,False
25890,False


In [9]:
char_cols_lst = list(X_train_dep.select_dtypes(exclude=['int64','float64','bool']).columns)
char_cols_lst

['job',
 'marital',
 'education',
 'default',
 'housing',
 'loan',
 'contact',
 'month']

In [10]:
X_train_dep

,age,job,marital,education,default,balance,housing,loan,contact,day,...,month_aug,month_dec,month_feb,month_jan,month_jul,month_jun,month_mar,month_may,month_nov,month_oct
39336,30,admin,single,secondary,no,2,yes,no,telephone,18,...,0,0,0,0,0,0,0,1,0,0
25184,36,services,single,secondary,no,1482,yes,no,cellular,18,...,0,0,0,0,0,0,0,0,1,0
34484,35,blue-collar,married,primary,no,147,yes,no,cellular,5,...,0,0,0,0,0,0,0,1,0,0
25237,33,management,married,secondary,no,480,yes,no,cellular,18,...,0,0,0,0,0,0,0,0,1,0
16170,46,blue-collar,married,secondary,no,209,yes,no,cellular,22,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2072,31,blue-collar,married,primary,no,0,yes,yes,unknown,12,...,0,0,0,0,0,0,0,1,0,0
27318,39,management,married,tertiary,no,3216,no,no,cellular,21,...,0,0,0,0,0,0,0,0,1,0
32879,33,blue-collar,single,primary,no,557,yes,no,cellular,17,...,0,0,0,0,0,0,0,0,0,0
2577,31,admin,single,secondary,no,513,yes,no,unknown,13,...,0,0,0,0,0,0,0,1,0,0


In [11]:
X_train_dep.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,...,month_aug,month_dec,month_feb,month_jan,month_jul,month_jun,month_mar,month_may,month_nov,month_oct
39336,30,admin,single,secondary,no,2,yes,no,telephone,18,...,0,0,0,0,0,0,0,1,0,0
25184,36,services,single,secondary,no,1482,yes,no,cellular,18,...,0,0,0,0,0,0,0,0,1,0
34484,35,blue-collar,married,primary,no,147,yes,no,cellular,5,...,0,0,0,0,0,0,0,1,0,0
25237,33,management,married,secondary,no,480,yes,no,cellular,18,...,0,0,0,0,0,0,0,0,1,0
16170,46,blue-collar,married,secondary,no,209,yes,no,cellular,22,...,0,0,0,0,1,0,0,0,0,0


### Label Encode

In [13]:
char_cols_lst

['job',
 'marital',
 'education',
 'default',
 'housing',
 'loan',
 'contact',
 'month']

In [14]:
for categ_feat in char_cols_lst:
    label_encode(X_train_dep,categ_feat)

In [15]:
X_train_dep[char_cols_lst].head()

,job,marital,education,default,housing,loan,contact,month
39336,0,2,1,0,1,0,1,8
25184,7,2,1,0,1,0,0,9
34484,1,1,0,0,1,0,0,8
25237,4,1,1,0,1,0,0,9
16170,1,1,1,0,1,0,0,5


In [16]:
for categ_feat in char_cols_lst:
    label_encode(X_test_dep,categ_feat)

In [17]:
X_test_dep[char_cols_lst].head()

,job,marital,education,default,housing,loan,contact,month
13130,9,1,1,0,0,1,0,5
11127,7,0,1,0,0,0,2,6
17899,6,0,2,0,0,0,0,5
39652,4,2,2,0,1,0,0,8
25890,2,1,2,0,1,0,0,9


In [18]:
X_train_dep.columns

Index(['age', 'job', 'marital', 'education', 'default', 'balance', 'housing',
       'loan', 'contact', 'day', 'month', 'duration', 'campaign',
       'job_blue-collar', 'job_entrepreneur', 'job_housemaid',
       'job_management', 'job_retired', 'job_self-employed', 'job_services',
       'job_student', 'job_technician', 'job_unemployed', 'job_unknown',
       'marital_married', 'marital_single', 'education_secondary',
       'education_tertiary', 'education_unknown', 'default_yes', 'housing_yes',
       'loan_yes', 'contact_telephone', 'contact_unknown', 'month_aug',
       'month_dec', 'month_feb', 'month_jan', 'month_jul', 'month_jun',
       'month_mar', 'month_may', 'month_nov', 'month_oct'],
      dtype='object')

### drop dummies 

In [19]:
dummies_lst = ['job_blue-collar',
 'job_entrepreneur',
 'job_housemaid',
 'job_management',
 'job_retired',
 'job_self-employed',
 'job_services',
 'job_student',
 'job_technician',
 'job_unemployed',
 'job_unknown',
 'marital_married',
 'marital_single',
 'education_secondary',
 'education_tertiary',
 'education_unknown',
 'default_yes',
 'housing_yes',
 'loan_yes',
 'contact_telephone',
 'contact_unknown',
 'month_aug',
 'month_dec',
 'month_feb',
 'month_jan',
 'month_jul',
 'month_jun',
 'month_mar',
 'month_may',
 'month_nov',
 'month_oct']

In [22]:

X_train_dep_dd = X_train_dep.drop(dummies_lst, axis= 1)



In [23]:
X_train_dep_dd.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 28000 entries, 39336 to 22418
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   age        28000 non-null  int64
 1   job        28000 non-null  int32
 2   marital    28000 non-null  int32
 3   education  28000 non-null  int32
 4   default    28000 non-null  int32
 5   balance    28000 non-null  int64
 6   housing    28000 non-null  int32
 7   loan       28000 non-null  int32
 8   contact    28000 non-null  int32
 9   day        28000 non-null  int64
 10  month      28000 non-null  int32
 11  duration   28000 non-null  int64
 12  campaign   28000 non-null  int64
dtypes: int32(8), int64(5)
memory usage: 2.1 MB


In [24]:
X_train_dep_dd.shape

(28000, 13)

In [26]:
X_test_dep_dd = X_test_dep.drop(columns = dummies_lst)
X_test_dep_dd.shape

(12000, 13)

In [27]:
X_test_dep_dd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12000 entries, 13130 to 7447
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   age        12000 non-null  int64
 1   job        12000 non-null  int32
 2   marital    12000 non-null  int32
 3   education  12000 non-null  int32
 4   default    12000 non-null  int32
 5   balance    12000 non-null  int64
 6   housing    12000 non-null  int32
 7   loan       12000 non-null  int32
 8   contact    12000 non-null  int32
 9   day        12000 non-null  int64
 10  month      12000 non-null  int32
 11  duration   12000 non-null  int64
 12  campaign   12000 non-null  int64
dtypes: int32(8), int64(5)
memory usage: 937.5 KB


In [28]:
y_train_dep_df.head()

,y_label
39336,False
25184,False
34484,False
25237,False
16170,False


In [29]:
y_test_dep_df.head()

,y_label
13130,False
11127,True
17899,True
39652,False
25890,False


In [30]:
features= ['age', 'job', 'marital', 'education', 'default',
       'balance', 'housing', 'loan', 'contact', 'day', 'month', 'duration',
       'campaign']

# Best model mod_2

In [38]:
mod_2 = class_rs(
#     model=  LogisticRegression(solver = 'saga', class_weight='balanced', max_iter=500),
model = GradientBoostingClassifier(),
x_train = X_train_dep[features], 
x_test =  X_test_dep[features],
y_train = y_train_dep_df.y_label,
y_test =  y_test_dep_df.y_label,
params ={'max_depth':[1,2],'n_estimators':[1000, 2000, 3000, 4000, 5000], 'learning_rate':[0.01,0.001],
         'random_state':[0]},
    # params = {'penalty':['l2'],'C':[0.0001, 0.01]} # ridge
#     params = {'penalty':['l1'],'C':[0.0001, 0.01]} # lasso
    # params = {'penalty':['elasticnet'],'C':[0.0001, 0.01], 'l1_ratio':[0.5]}  # elastic net
splits = 5,
scoring= "accuracy",
logistic_reg = False,
tree_based = True)

best params:  {'random_state': 0, 'n_estimators': 2000, 'max_depth': 2, 'learning_rate': 0.01}
score:  0.9395714285714286
  
test score:  0.9363333333333334
confusion matrix train
[[25576   397]
 [ 1295   732]]
 
confusion matrix test
[[10926   205]
 [  559   310]]
 
 
train roc_auc_score:  0.9441352240270915
 
test roc_auc_score:  0.9365803049135834
 
Training KS statistic
KstestResult(statistic=0.7687659821911758, pvalue=0.0)
 
Test KS statistic
KstestResult(statistic=0.7458823619415148, pvalue=0.0)
 
     features  importances
0    duration     0.742858
1       month     0.117443
2         age     0.038783
3     contact     0.036218
4         day     0.036100
5     housing     0.015399
6    campaign     0.003982
7     balance     0.003466
8   education     0.003277
9     marital     0.001418
10        job     0.000529
11       loan     0.000529
12    default     0.000000


In [32]:
mod_3 = class_rs(
#     model=  LogisticRegression(solver = 'saga', class_weight='balanced', max_iter=500),
model = GradientBoostingClassifier(),
x_train = X_train_dep[features], 
x_test =  X_test_dep[features],
y_train = y_train_dep_df.y_label,
y_test =  y_test_dep_df.y_label,
params ={'max_depth':[1,2],'n_estimators':[1000, 2000, 3000, 4000, 5000], 'learning_rate':[0.01,0.001],
         'random_state':[0]},
    # params = {'penalty':['l2'],'C':[0.0001, 0.01]} # ridge
#     params = {'penalty':['l1'],'C':[0.0001, 0.01]} # lasso
    # params = {'penalty':['elasticnet'],'C':[0.0001, 0.01], 'l1_ratio':[0.5]}  # elastic net
splits = 5,
scoring= "accuracy",
logistic_reg = False,
tree_based = True)

best params:  {'random_state': 0, 'n_estimators': 3000, 'max_depth': 2, 'learning_rate': 0.01}
score:  0.9362142857142857
  
test score:  0.93475
confusion matrix train
[[25566   407]
 [ 1379   648]]
 
confusion matrix test
[[10949   182]
 [  601   268]]
 
 
train roc_auc_score:  0.9480381518730572
 
test roc_auc_score:  0.9413737269895632
 
Training KS statistic
KstestResult(statistic=0.7869497357232438, pvalue=0.0)
 
Test KS statistic
KstestResult(statistic=0.7648888811237321, pvalue=0.0)
 
     features  importances
0    duration     0.701870
1       month     0.145410
2         day     0.042813
3         age     0.042501
4     contact     0.036672
5     housing     0.015340
6     balance     0.004282
7   education     0.004031
8    campaign     0.003369
9     marital     0.001726
10        job     0.001416
11       loan     0.000571
12    default     0.000000


In [36]:
cm_infor(mod_3)

recall =  0.3084004602991945
specificity =  0.983649267810619
accuracy =  0.93475
precision =  0.5955555555555555
F1 score =  0.40636846095526913


In [33]:
mod_2_final = class_rs(
#     model=  LogisticRegression(solver = 'saga', class_weight='balanced', max_iter=500),
model = GradientBoostingClassifier(),
x_train = X_train_dep[features], 
x_test =  X_test_dep[features],
y_train = y_train_dep_df.y_label,
y_test =  y_test_dep_df.y_label,
params ={'max_depth':[2],'n_estimators':[ 2000], 'learning_rate':[0.01],
         'random_state':[0]},
    # params = {'penalty':['l2'],'C':[0.0001, 0.01]} # ridge
#     params = {'penalty':['l1'],'C':[0.0001, 0.01]} # lasso
    # params = {'penalty':['elasticnet'],'C':[0.0001, 0.01], 'l1_ratio':[0.5]}  # elastic net
splits = 5,
scoring= "accuracy",
logistic_reg = False,
tree_based = True)

C:\Users\dgarb\anaconda3_reinstall\lib\site-packages\sklearn\model_selection\_search.py:305: UserWarning: The total space of parameters 1 is smaller than n_iter=10. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


best params:  {'random_state': 0, 'n_estimators': 2000, 'max_depth': 2, 'learning_rate': 0.01}
score:  0.93425
  
test score:  0.9348333333333333
confusion matrix train
[[25562   411]
 [ 1430   597]]
 
confusion matrix test
[[10958   173]
 [  609   260]]
 
 
train roc_auc_score:  0.9444772740452207
 
test roc_auc_score:  0.9383637523585371
 
Training KS statistic
KstestResult(statistic=0.7791045807483545, pvalue=0.0)
 
Test KS statistic
KstestResult(statistic=0.7567740970360408, pvalue=0.0)
 
     features  importances
0    duration     0.717854
1       month     0.136033
2         age     0.040540
3         day     0.038780
4     contact     0.037320
5     housing     0.015579
6   education     0.003856
7     balance     0.003395
8    campaign     0.003381
9     marital     0.001662
10        job     0.001072
11       loan     0.000528
12    default     0.000000


In [35]:
cm_infor(mod_2_final)

recall =  0.2991944764096663
specificity =  0.9844578205013027
accuracy =  0.9348333333333333
precision =  0.6004618937644342
F1 score =  0.39938556067588327
